# Bag-of-WordsとLightGBMで恋愛コラムの本文からライターを判定する

## 概要

- [Googirl](https://googirl.jp/)という女性向け恋愛コラムのWebサイトがあります。
- 記事ごとに書いているライターが異なります。本文をMeCabで形態素解析して作ったBag-of-Words (BoW) を特徴量としてライターを分類するLightGBMモデルを作ってみました。
  - コラムの総数が多い上位5人のライターに絞っています。記事数は4000件程度、1本の記事は約500文字程度です。
  - 記事のBoWを特徴量にした5クラス分類タスクです。
  - LightGBMはoptuna.integration.lightgbmを使いました。
- 予測精度は95%程度でした。

## データ

- 対象は、Googirlの[恋愛・結婚カテゴリ](https://googirl.jp/renai/)のうち、2017/1/1～2021/6/15に書かれたコラムです。そのうち、投稿総数上位5位のライターのみを取り出しました。
  - ただし、続き物のコラムは最初の1本のみを取り出しました。
    - vol.1～vol.9や第1位～第9位、前編・後編のように、同じライターが同じテーマを複数の記事に分割していることが多いです。この場合、vol.1、第1位、前編のみをデータに残し、それ以外の記事は含めていないということです。というのも、これらの記事は同じテーマであるため、ある意味ライターを予測できるのが当たり前だからです。
  - 全部で4302件の記事です。
- Googirlの恋愛・結婚カテゴリをスクレイピングし、タイトル（使わない）、本文、ライター名を列に持つpd.DataFrameにしました。
  - 他にもカテゴリはありますが、同じカテゴリの文章を分類する方が難易度が高く面白いと思ったので、恋愛・結婚カテゴリに絞りました。
  - 十分なスリープを挟みました。


In [13]:
import MeCab
import neologdn
import pandas as pd
import numpy as np
from datetime import datetime
import optuna.integration.lightgbm as lgb
from optuna.logging import set_verbosity
from lightgbm import Dataset, early_stopping, log_evaluation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tqdm import tqdm
import time
from customized_tagger import CustomizedTagger

# 読み込みと前処理 ---------------------------------------------------------------------
freq_author=5
googirl_article=pd.read_csv("../data/googirl/googirl_article_renai_and_marriage_20210615.csv")
googirl_article=(
    googirl_article
    .assign(date=lambda d: pd.to_datetime(d.date))
    .loc[lambda d: d.date>=datetime(2017,1,1)]
    .assign(
        title=lambda d: d.title.map(neologdn.normalize),
    ).loc[
        # [vV]ol. 1や前編、第1位以外の連番のタイトルの記事を削除する
        (lambda d: 
            ~d.title.str.contains(r"[vV]ol\. ?[2-9]") &
            ~d.title.str.contains("[中後]編") &
            ~d.title.str.contains("第[2-9]位")
        )
    ]
)

# ライター別の記事数
n_by_author=(
    googirl_article.author
    .value_counts(ascending=False)
    # indexにあるauthorを列として戻すため
    .reset_index()
    .rename(columns={"index":"author","author":"n"})
)
print(n_by_author)
used_authors=n_by_author.author.head(freq_author).to_list()
print(used_authors)

    author     n
0     松はるな  1544
1        和   729
2     小林ユリ   682
3     Waxy   680
4       美佳   667
..     ...   ...
78   メカセツコ     1
79  羅生門の老婆     1
80  きゃおりーな     1
81     イッヌ     1
82    Risa     1

[83 rows x 2 columns]
['松はるな', '和', '小林ユリ', 'Waxy', '美佳']


## 前処理

- 以下を削除しました。
  - htmlタグ
  - 下三角形やかぎかっこの前の中黒
    - ディバイダー的に使われているものです。
  - 記事内に出てくるインタビューした男性の年齢や職種が（）内に書いてあります。これを削除しました。
    - 削除しないとBoWを作る上で結構なノイズになることが後で分かりました。
- 正規化はneologd.normalizeを用いました。
  - NFKC正規化にいくつかの表記の揺れを直してくれる便利なライブラリです。

In [2]:
article=(
    googirl_article
    .loc[lambda d: d.author.isin(used_authors)]
    # urlの最後のサブディレクトリ（https://hoge.com/fuga/piyo/のpiyo）を取り出して一意なキーとして使う
    .assign(id_url=lambda d: d.url.str.extract("([^/]+)(?=/$)",expand=True))
    # 連番を振る
    .assign(id=lambda d: pd.RangeIndex(start=0,stop=len(d),step=1).astype(str))
    # .assign(text=lambda d: d.title)
    .assign(
        text=lambda d: d.text.str.replace(r"[\(（][0-9]+.*[）\)]","",regex=True)
                                .str.replace(r"\[\[\[p\]\]\]","",regex=True)
                                .str.replace("▽","",regex=True)
                                .str.replace("・(?=「)","",regex=True)
                                .map(neologdn.normalize)
     )
     .reset_index()
     .filter(["id","id_url","url","date","title","author","text"])
)

df=article.filter(["id","id_url","author","title","text"])
print(df.head(5))

  id     id_url author                             title  \
0  0  210614009     美佳              12星座別!彼の究極の欠点とは?【前編】   
1  1  210614006   松はるな    何が違うの?「彼女にしたい女」と「友達止まりの女」の3つの差   
2  2  210614019   Waxy  直感って侮れない!「この人と結婚するかも…」が当たったケース4つ   
3  3  210613006   松はるな     運命かも!男性が「相性のよさ」を感じて惚れる女性の言動3つ   
4  4  210613015   Waxy   幼稚で付き合いきれない!「ナルシストだな…」と思う彼の言動4つ   

                                                text  
0  誰にでも一つくらいは欠点があるもの。付き合う上では、相手の欠点を受け入れられるかどうかがかな...  
1  いい感じに仲よくなってきたな……と思いきや、「俺たちっていい友達だよな!」とまさかの友達発言...  
2  長いあいだ婚活を続けてもなかなかうまくいかない人もいれば、短期間でサクッと結婚を決めるような...  
3  「この人とは相性がいいかも!」と感じる相手とは関係が発展しやすいし、付き合うことになっても特...  
4  見た目は一人前の立派な大人の男性。でも中身は幼稚で、いつも周りから注目されていないと不満……...  


## 形態素解析と特徴量のBoW作成

- Windows 10 + MeCab 32bit (v0.996) + [mecab](https://pypi.org/project/mecab/) (v.0.996.3) を用いました。
  - 辞書はデフォルトのIPA辞書と2020/8/20（本記事を執筆時で最新）のmecab-ipadic-NEologdです。
  - MeCabのPythonバインディングは、私の環境ではmecab-python3のインストールに失敗したのでmecabを用いています。
- 特徴量
  - 名詞のみ用いています。
  - ストップワードに[Slothlib](http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt)を用いています。
  - BoWは0か1かの2値にしています。
  - scikit-learnのCountVectorizerのmin_dfを0.001にしています。
    - つまり、4302記事の中の4記事以下でしか出現しない単語を削除しています。
    - min_dfを0にすると13136個の特徴量（＝13136次元）ですが、0.001にしたことで4568次元まで減りました。
- CountVectorizerには、原型のまま分かち書きをして名詞のみ残したものを渡しています。
  - 実装のコツとして、MeCab.Tagger("-Ochasen");MeCab.Tagger.parseToNodeの結果をpd.DataFrameで返すラッパーの自作クラスを作っています（末尾のセルを参照）。このDataFrameから品詞が名詞のもののみを抽出して、その原型の空白を挟んでくっつけています。
- 余談ですが私はRとPythonの両方でMeCabを用いています。Windows環境でのMeCabは、RではShift-JIS、PythonではUTF-8の辞書を用いるため、デフォルトのipadicとNEologdともにShift-JISとUTF-8の両方の辞書を作成しています。MeCab.Tagger()は、MeCab.Tagger("-r .mecabrcのパス")で使用する.mecabrcを指定できますので、UTF-8版の辞書を記載した.mecabrcを引数に与えることでUTF-8の辞書を使えるようにしています。

In [3]:
# 形態素解析と特徴量を作る ---------------------------------------------------------------------
ct=CustomizedTagger("-Ochasen -r ../utf8.mecabrc")
texts=df.text.to_list()
# 形態素解析し、名詞のみを取り出して空白区切りの分かち書きにする
# [np.array(["吾輩 猫 だ"],np.array(["名前 まだ ない"])]のようなもの
wakati=[np.array(ct.wakati_extract(j,["名詞"])) for i,j in enumerate(tqdm(texts))]
# np.array(["吾輩 猫","名前 まだ ない"])にする
wakati=np.concatenate(wakati)

# Bag-of-Wordを作る
stop_words=(
    pd.read_csv("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt",header=None)
    .set_axis(["word"],axis=1)
    .word
    .to_list()
)
cv=CountVectorizer(min_df=0.001,binary=True,stop_words=stop_words)
cv_transform=cv.fit_transform(wakati)
df_x=pd.DataFrame(cv_transform.toarray(),columns=cv.get_feature_names_out())
x=np.array(df_x)

# 目的変数のauthor列をラベルエンコーティングする
le=LabelEncoder()
df_y=df.assign(target=lambda d: le.fit_transform(d.author))
y=np.array(df_y.target)

print(df_x.iloc[:,0:10].head(5))

100%|██████████| 4302/4302 [00:07<00:00, 548.02it/s]


   100  100年  10代  10分  10年  12月  180度  1か月  1人  1回
0    0     0    0    0    0    0     0    0   0   0
1    0     0    0    0    0    0     0    0   0   0
2    0     0    0    0    0    0     0    0   0   0
3    0     0    0    0    0    0     0    0   0   0
4    0     0    0    0    0    0     0    0   0   0


## 学習と評価

- Nested CVを用いています。
  - 外側のCV（学習と予測のFold）と内側のCV（学習の中のハイパーパラメータの調整のFold）のいずれも、5-FoldのStratified Foldとしています。
    - Stratifiedなのはラベルが不均衡だからです。
  - optunaのlightGBM実装を用いました。ハイパーパラメータ探索はoptunaのベイズ最適化です。
  - i9-9900Kの環境で1～2時間で終わっていたと思います。メモリは4GB程度使っていました。

In [ ]:
# 学習と予測 ---------------------------------------------------------------------
params = {
    'objective': 'multiclass',
    'num_class': freq_author,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'deterministic': True,
    'force_row_wise': True
    }
set_verbosity(1)
callbacks=[
    early_stopping(stopping_rounds=20),
    log_evaluation(period=20)
]

start_time=time.time()
pred_all=[]
true_all=[]
test_idx_all=[]
# 外側のCV（学習データと評価データ）
fold=StratifiedKFold(n_splits=5,shuffle=True,random_state=1234)
for train_idx, test_idx in fold.split(x,y):
    x_train=x[train_idx]
    y_train=y[train_idx]
    x_test=x[test_idx]
    y_test=y[test_idx]

    lgb_train=lgb.Dataset(x_train,y_train)
    lgb_test=lgb.Dataset(x_test,y_test)
    # 内側のCV（学習データのうち、ハイパーパラメータの学習データと評価データに分ける）
    inner_fold=StratifiedKFold(n_splits=5,shuffle=True,random_state=1234)
    tuner_cv = lgb.LightGBMTunerCV(
        params,
        lgb_train,
        num_boost_round=1000,
        folds=inner_fold,
        callbacks=callbacks,
        optuna_seed=1234,
        return_cvbooster=True
    )
    tuner_cv.run()
    # 最もよいモデルを取り出し、保存してからそのモデルをロードし、それでテストデータを予測する
    best_model=tuner_cv.get_best_booster()
    best_model.save_model("lgb_model.txt")
    best_model=lgb.Booster(model_file="lgb_model.txt")
    pred=best_model.predict(x_test)
    pred=np.argmax(pred, axis=1)
    
    pred_all.append(pred)
    true_all.append(y_test)
    test_idx_all.append(test_idx)
    print(confusion_matrix(pred,y_test))
    print(accuracy_score(pred,y_test))

time.time()-start_time

pred_all_concat=np.concatenate(pred_all)
true_all_concat=np.concatenate(true_all)
test_idx_all_concat=np.concatenate(test_idx_all)
df_pred=pd.DataFrame(zip(test_idx_all_concat,pred_all_concat),columns=["id","pred"]).sort_values("id").assign(id=lambda d: d.id.astype(str))

# 予測結果を結合したdf
df_res=(
    pd.merge(df,df_pred,on="id",how="inner")
    .assign(pred=lambda d: le.inverse_transform(d.pred))
)


## 予測結果

In [ ]:
# 混同行列と予測精度を学習したモデルから求めるならこれを使うが、
# 本notebookでは事前に回したコードで求めている予測結果を読み込むためコメントアウト

# conf_mat=pd.DataFrame(
#     confusion_matrix(pred_all_concat,true_all_concat),
#     # column,indexともに0-4なので
#     columns=le.inverse_transform(np.unique(y)),
#     index=le.inverse_transform(np.unique(y)),
# )
# print(conf_mat)
# print(classification_report(df_res.author,df_res.pred)
# print("accuracy_score: " + str(accuracy_score(pred_all_concat,true_all_concat)))
# print(df_res.head(5))

In [15]:
df_res=pd.read_csv("../googirl_classify_result.csv")
confusion_matrix(df_res.author,df_res.pred)

conf_mat=pd.DataFrame(
    confusion_matrix(df_res.author,df_res.pred),
    # column,indexともに0-4なので
    columns=le.inverse_transform(np.unique(y)),
    index=le.inverse_transform(np.unique(y)),
)
print(conf_mat)
print(classification_report(df_res.author,df_res.pred))
print("accuracy_score: " + str(accuracy_score(df_res.author,df_res.pred)))

      Waxy    和  小林ユリ  松はるな   美佳
Waxy   650    7    14     2    7
和        1  692     2    12   22
小林ユリ    10   17   627    22    6
松はるな     4   12     4  1518    6
美佳       5   27     6    13  616
              precision    recall  f1-score   support

        Waxy       0.97      0.96      0.96       680
           和       0.92      0.95      0.93       729
        小林ユリ       0.96      0.92      0.94       682
        松はるな       0.97      0.98      0.98      1544
          美佳       0.94      0.92      0.93       667

    accuracy                           0.95      4302
   macro avg       0.95      0.95      0.95      4302
weighted avg       0.95      0.95      0.95      4302

accuracy_score: 0.9537424453742446


In [12]:
print(df_res.head(5))

   Unnamed: 0  id     id_url                                  url        date  \
0           0   0  210614009  https://googirl.jp/renai/210614009/  2021-06-14   
1           1   1  210614006  https://googirl.jp/renai/210614006/  2021-06-14   
2           2   2  210614019  https://googirl.jp/renai/210614019/  2021-06-14   
3           3   3  210613006  https://googirl.jp/renai/210613006/  2021-06-13   
4           4   4  210613015  https://googirl.jp/renai/210613015/  2021-06-13   

                              title author  \
0              12星座別!彼の究極の欠点とは?【前編】     美佳   
1    何が違うの?「彼女にしたい女」と「友達止まりの女」の3つの差   松はるな   
2  直感って侮れない!「この人と結婚するかも…」が当たったケース4つ   Waxy   
3     運命かも!男性が「相性のよさ」を感じて惚れる女性の言動3つ   松はるな   
4   幼稚で付き合いきれない!「ナルシストだな…」と思う彼の言動4つ   Waxy   

                                                text  pred  
0  誰にでも一つくらいは欠点があるもの。付き合う上では、相手の欠点を受け入れられるかどうかがかな...    美佳  
1  いい感じに仲よくなってきたな……と思いきや、「俺たちっていい友達だよな!」とまさかの友達発言...  松はるな  
2  長いあいだ婚活を続けてもなかなかうまくいかない人もいれば、短期間でサクッと結婚を決めるような... 

## 感想

- 同じカテゴリの文章なので分類が難しいかと思いましたが、思いの外高い精度が出ました。
  - 実はライターによって使っている単語が若干違います。それでも人間が読んで見分けるのは難しいです。
- BoWは古典的な手法ですが、このようにタスクによっては強力です。
- コラムに目を通して、このデータセットに適した前処理を考える所が一番大変でした。

## （参考）MeCabの出力のパースをする自作クラス

In [ ]:
import MeCab
import pandas as pd
from typing import List

class CustomizedTagger(MeCab.Tagger):
    cols=["surface","pos","pos1","pos2","pos3","form1","form2","base","yomi","hatsuon"]

    def parse(self,text: str) -> pd.DataFrame:
        res=[]
        node=self.parseToNode(text)
        while node:
            surface=[node.surface]
            feature=node.feature.split(",")
            if (feature[0]!="BOS/EOS"):
                res.append([*surface,*feature])
            node=node.next
        res_df=pd.DataFrame(res,columns=self.cols)
        return res_df

    # 分かち書きのうち、posに指定した品詞のみ原型にして取り出す
    def wakati_base_extract(self,text: str,pos: List[str]) -> List[str]:
        parsed=self.parse(text)
        extracted_list=parsed.loc[lambda d: d.pos.isin(pos)].base.to_list()
        res=" ".join(extracted_list)
        res=[res]
        return res